#**Mineração e Análise de Dados do Linkedin**

1. Baixe seus arquivos do Linkedin [aqui](https://https://www.linkedin.com/mypreferences/d/download-my-data). Você receberá os arquivos por email em 24h.
2. Baixe e descompacte o arquivo recebido, busque os arquivos indicados ao longo do código.
3. Usaremos os arquivos de conexões e convites, faremos a limpeza dos dados e em seguida iremos anonimizar.

## Bases de dados em .csv

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
!pip install --upgrade plotly

### Conexões

In [ ]:
conexoes = pd.read_csv('Connections.csv')

In [ ]:
conexoes.columns

In [ ]:
conexoes.shape

In [ ]:
conexoes.head()

In [ ]:
conexoes.drop(labels = ['Email Address'], axis=1, inplace=True)

In [ ]:
conexoes.head()

In [ ]:
conexoes.isna().sum()

In [ ]:
conexoes.shape

In [ ]:
conexoes.dropna(inplace=True)
conexoes.shape

In [ ]:
conexoes.isna().sum()

In [ ]:
conexoes['Full Name'] = conexoes['First Name'] + ' ' + conexoes['Last Name']
conexoes.head()

### Convites

In [ ]:
convites = pd.read_csv('Invitations.csv')

In [ ]:
convites.columns

In [ ]:
convites.shape

In [ ]:
convites.head(10)

In [ ]:
convites.isna().sum()

In [ ]:
convites[convites['Message'].notnull()].head(10)

In [ ]:
convites.shape

In [ ]:
convites = convites[convites['Direction'] == 'INCOMING']
convites.shape

In [ ]:
np.unique(convites['Direction'])

In [ ]:
convites['Company'] = ''
convites.head()

In [ ]:
conexoes[conexoes['Full Name'] == 'Milos Naflas']['Company'].values[0]

In [ ]:
len(convites)

In [ ]:
convites = convites.reset_index(drop=True)
convites

In [ ]:
for i in range(0, len(convites)):
  #print(i, convites['From'][i])
  try:
    company = conexoes[conexoes['Full Name'] == convites['From'][i]]['Company'].values[0]
    #print(company)
    convites['Company'][i] = company
  except:
    continue

In [ ]:
convites

In [ ]:
convites.drop(labels=['From', 'To', 'Sent At', 'Message', 'Direction'], axis=1, inplace=True)
convites

In [ ]:
convites.to_csv('convites.csv')

### Geração de dados fake

In [ ]:
conexoes.drop(labels = ['First Name', 'Last Name', 'Full Name'], axis = 1, inplace = True)
conexoes

In [ ]:
!pip install faker

In [ ]:
from faker import Faker

In [ ]:
fake = Faker()

In [ ]:
fake.name()

In [ ]:
conexoes['Full Name'] = ''
conexoes.head()

In [ ]:
conexoes = conexoes.reset_index(drop=True)

In [ ]:
for i in range(0, len(conexoes)):
  conexoes['Full Name'][i] = fake.name()

In [ ]:
conexoes

In [ ]:
conexoes.to_csv('conexoes.csv')

### Mensagens

In [ ]:
mensagens = pd.read_csv('messages.csv')

In [ ]:
mensagens.columns

In [ ]:
mensagens.shape

In [ ]:
mensagens.head(1)

In [ ]:
mensagens = mensagens['CONTENT']

In [ ]:
mensagens

In [ ]:
mensagens.isna().sum()

In [ ]:
mensagens.dropna(inplace=True)
mensagens.shape

In [ ]:
mensagens.to_csv('mensagens.csv')

## Conexões



### Número de conexões por período

In [ ]:
import datetime
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
!pip install --upgrade plotly

In [ ]:
conexoes = pd.read_csv('conexoes.csv')
conexoes.drop(labels = ['Unnamed: 0'], axis = 1, inplace=True)

In [ ]:
conexoes.head()

In [ ]:
def converte_data(data):
  return datetime.datetime.strptime(data, '%d %b %Y').strftime('%Y-%m-%d')

In [ ]:
conexoes['Connected On'] = conexoes['Connected On'].apply(converte_data)

In [ ]:
conexoes.head()

In [ ]:
grafico = px.scatter(conexoes, x = 'Full Name', y = 'Connected On')
grafico.show()

In [ ]:
conexoes.groupby(by = 'Connected On').count()

In [ ]:
grafico = px.line(conexoes.groupby(by = 'Connected On').count(), title = 'Novas conexões por data')
grafico.show()

In [ ]:
def converte_data_mes(data):
  return datetime.datetime.strptime(data, '%Y-%m-%d').strftime('%m')

In [ ]:
conexoes['Connected Month'] = conexoes['Connected On'].apply(converte_data_mes)
conexoes

In [ ]:
conexoes.groupby(by = 'Connected Month').count()

In [ ]:
grafico = px.line(conexoes.groupby(by = 'Connected Month').count(), title = 'Novas conexões por mês')
grafico.show()

In [ ]:
def converte_data_ano(data):
  return datetime.datetime.strptime(data, '%Y-%m-%d').strftime('%Y')

In [ ]:
conexoes['Connected Year'] = conexoes['Connected On'].apply(converte_data_ano)
conexoes

In [ ]:
conexoes.groupby(by = 'Connected Year').count()

In [ ]:
grafico = px.line(conexoes.groupby(by = 'Connected Year').count(), title = 'Novas conexões por ano')
grafico.show()

### Empresas

In [ ]:
np.unique(conexoes['Company'], return_counts=True)

In [ ]:
conexoes.shape

In [ ]:
len(np.unique(conexoes['Company']))

In [ ]:
grafico = px.histogram(conexoes['Company'])
grafico.show()

In [ ]:
grafico = px.treemap(conexoes, path=['Company', 'Position', 'Full Name'])
grafico.show()

### Cargos

In [ ]:
np.unique(conexoes['Position'], return_counts=True)

In [ ]:
conexoes.shape

In [ ]:
len(np.unique(conexoes['Position']))

In [ ]:
grafico = px.histogram(conexoes['Position'])
grafico.show()

In [ ]:
def altera_cargo(cargo):
  novo_cargo = cargo
  if cargo == 'Data Scientist':
    novo_cargo = 'Cientista de Dados'
  elif cargo == 'Cientista de dados':
    novo_cargo = 'Cientista de Dados'
  return novo_cargo

In [ ]:
conexoes['Position'] = conexoes['Position'].apply(altera_cargo)

In [ ]:
len(conexoes[conexoes['Position'] == 'Cientista de Dados'])

In [ ]:
grafico = px.treemap(conexoes, path = ['Position', 'Company', 'Full Name'])
grafico.show()

### Agrupamento dos cargos

#### Distância de edição (Levenshtein)

- https://pt.wikipedia.org/wiki/Dist%C3%A2ncia_Levenshtein#:~:text=Em%20teoria%20da%20informa%C3%A7%C3%A3o%2C%20a,transformar%20um%20string%20no%20outro.
- Quantas inserções, exclusões e substituições são necessárias para converter uma string em outra
- Exemplo: pai e cai: 1 substituição

In [ ]:
# pip install nltk
import nltk

In [ ]:
from nltk.metrics.distance import edit_distance

In [ ]:
edit_distance('rain', 'shine')

In [ ]:
edit_distance('pai', 'cai')

In [ ]:
edit_distance('Analista de Dados', 'Analista de dados')

In [ ]:
edit_distance('Consultant', 'Consultor')

In [ ]:
edit_distance('Cientista de Dados', 'Data Scientist')

In [ ]:
edit_distance('Cientista de Dados', 'Cientista de Dados')

#### Similaridade n-grama

- n-grama é uma forma de expressar cada sequência consecutiva possível de n tokens de um texto
- Computamos os "bigramas" para todos os tokens de duas strings e pontuamos a similaridade contando o número de bigramas comuns
- Outras métricas para avaliação: https://tedboy.github.io/nlps/generated/generated/nltk.BigramAssocMeasures.html

In [ ]:
'analista de business intelligence'.split()

In [ ]:
bigramas_cargo1 = list(nltk.bigrams('analista de business intelligence'.split(), pad_right=True, pad_left=True))
bigramas_cargo1

In [ ]:
bigramas_cargo2 = list(nltk.bigrams('cientista de dados'.split(), pad_right=True, pad_left=True))
bigramas_cargo2

In [ ]:
bigramas_cargo3 = list(nltk.bigrams('analista de dados'.split(), pad_right=True, pad_left=True))
bigramas_cargo3

In [ ]:
set(['A', 'B', 'C']).intersection(set(['C', 'D', 'E']))

In [ ]:
# Cargo 1 x Cargo 2
len(set(bigramas_cargo1).intersection(set(bigramas_cargo2)))

In [ ]:
# Cargo 1 x Cargo 3
len(set(bigramas_cargo1).intersection(set(bigramas_cargo3)))

In [ ]:
# Cargo 2 x Cargo 3
len(set(bigramas_cargo2).intersection(set(bigramas_cargo3)))

In [ ]:
# Cargo 1 x Cargo 1
len(set(bigramas_cargo1).intersection(set(bigramas_cargo1)))

#### Distância de Jaccard

- Número de itens em comum dividido pelo número total de itens distintos em dois conjuntos
- Usa n-gramas
- https://en.wikipedia.org/wiki/Jaccard_index

In [ ]:
cargo1 = 'analista de business intelligence'.split()
cargo2 = 'cientista de dados'.split()
cargo3 = 'analista de dados'.split()

In [ ]:
cargo1, cargo2, cargo3

In [ ]:
# Cargo 1 x Cargo 3

In [ ]:
intersecao = set(cargo1).intersection(set(cargo3))
intersecao

In [ ]:
uniao = set(cargo1).union(set(cargo3))
uniao

In [ ]:
len(intersecao), len(uniao)

In [ ]:
(len(uniao) - len(intersecao)) / len(uniao)

In [ ]:
from nltk.metrics.distance import jaccard_distance

In [ ]:
# Cargo 1 x Cargo 3
jaccard_distance(set(cargo1), set(cargo3))

In [ ]:
# Cargo 1 x Cargo 2
jaccard_distance(set(cargo1), set(cargo2))

In [ ]:
# Cargo 2 x Cargo 3
jaccard_distance(set(cargo2), set(cargo3))

In [ ]:
# Cargo 1 x Cargo 1
jaccard_distance(set(cargo1), set(cargo1))

#### Agrupamento dos cargos

- Exemplos baseados em: https://github.com/mikhailklassen/Mining-the-Social-Web-3rd-Edition/blob/master/notebooks/Chapter%204%20-%20Mining%20LinkedIn.ipynb

##### Teste com poucos registros

In [ ]:
cargos_df = pd.DataFrame(columns=['Position'], data=['Programador', 'Cientista de Dados',
                                                     'Cientista de dados', 'Analista de business intelligence',
                                                     'Analista de dados', 'Consultant', 'Consultor',
                                                     'Consultora', 'Professor on-line', 'Professor universitário'])

In [ ]:
cargos_df

In [ ]:
todos_cargos = cargos_df['Position'].values
todos_cargos

In [ ]:
set(todos_cargos)

In [ ]:
for cargo1 in todos_cargos:
  print(cargo1)
  print('-------------')
  print('\n')
  for cargo2 in todos_cargos:
    print(cargo2, jaccard_distance(set(cargo1), set(cargo2)))
  print('\n')

In [ ]:
limite = 0.4
clusters = {}
for cargo1 in todos_cargos:
  clusters[cargo1] = []
  for cargo2 in todos_cargos:
    if cargo2 in clusters[cargo1] or cargo2 in clusters and cargo1 in clusters[cargo2]:
      continue

    distancia = jaccard_distance(set(cargo1), set(cargo2))
    if distancia <= limite:
      clusters[cargo1].append(cargo2)

In [ ]:
clusters

In [ ]:
clusters = [clusters[cargo] for cargo in clusters if len(clusters[cargo]) > 1]
clusters

##### Aplicação na base de dados

In [ ]:
todos_cargos = conexoes['Position'].values
len(todos_cargos)

In [ ]:
todos_cargos = set(todos_cargos)
len(todos_cargos)

In [ ]:
limite = 0.3
clusters = {}
for cargo1 in todos_cargos:
  clusters[cargo1] = []
  for cargo2 in todos_cargos:
    if cargo2 in clusters[cargo1] or cargo2 in clusters and cargo1 in clusters[cargo2]:
      continue

    distancia = jaccard_distance(set(cargo1), set(cargo2))
    if distancia <= limite:
      clusters[cargo1].append(cargo2)

In [ ]:
clusters = [clusters[cargo] for cargo in clusters if len(clusters[cargo]) > 1]
clusters, len(clusters)

##### Vinculação para os usuários

In [ ]:
for contato in range(0, len(conexoes)):
  print(conexoes['Position'][contato])

In [ ]:
cluster_contatos = {}
for cluster in clusters:
  #print(cluster)
  cluster_contatos[tuple(cluster)] = []
  for contato in range(0, len(conexoes)):
    if conexoes['Position'][contato] in cluster:
      cluster_contatos[tuple(cluster)].append(conexoes['Full Name'][contato])

In [ ]:
cluster_contatos

### Visualização do agrupamento

In [ ]:
from IPython.core.display import HTML
for cargos in cluster_contatos:
  lista_cargos = 'Lista de cargos no grupo: ' + ', '.join(cargos)
  #print(lista_cargos)

  termos = set(cargos[0].split())
  for palavras in cargos:
    termos.intersection_update(set(palavras.split()))
  if len(termos) == 0:
    termos = ['***Nenhum termo em comum****']
  termos_impressao = 'Termos comuns: ' + ', '.join(termos)

  display(HTML(f'<h3>{lista_cargos}</h3>'))
  display(HTML(f'<p>{termos_impressao}</p>'))
  display(HTML(f'<p>{"-" * 70}</p>'))
  display(HTML(f'<p><mark>{", ".join(cluster_contatos[cargos])}</mark></p>'))

  #print(lista_cargos)
  #print('\n' + termos_impressao)
  #print('-' * 70)
  #print('\n'.join(cluster_contatos[cargos]))
  #print()

### Exportação dos grupos para JSON

In [ ]:
from IPython.display import HTML
from IPython.core.display import display
import json

In [ ]:
cluster_contatos

In [ ]:
cluster_contatos[(('Systems Development Analyst','System Analyst','Senior Data Analyst'))]

In [ ]:
saida_json = {'name': 'Linkedin', 'children': []}
contador = 1
for grupos in cluster_contatos:
  #print(grupos)
  if contador > 10:
    break

  saida_json['children'].append({'name': ', '.join(grupos)[:20],
                                 'children': [{'name': contato} for contato in cluster_contatos[grupos]]})
  f = open('dados.json', 'w')
  f.write(json.dumps(saida_json, indent=1))
  f.close()

  contador += 1

In [ ]:
data = json.load(open('dados.json'))

In [ ]:
data

In [ ]:
valores = json.dumps(data)
valores

### Visualização com dendrograma

In [ ]:
# Exemplo baseado no livro Mineração de Dados da Web Social
visualizacao = """<!DOCTYPE html>
<meta charset='utf-8'>
<style>

.node circle {
  fill: #fff;
  stroke: steelblue;
  stroke-width: 1.5px;
}

.node {
  font: 10px sans-serif;
}

.link {
  fill: none;
  stroke: #ccc;
  stroke-width: 1.5px;
}

</style>
<body>
<script src='https://d3js.org/d3.v3.min.js'></script>
<script>

root = %s;
//j = JSON.parse(valores);

var width = 960,
    height = 2200;

var cluster = d3.layout.cluster()
    .size([height, width - 160]);

var diagonal = d3.svg.diagonal()
    .projection(function(d) { return [d.y, d.x]; });

var svg = d3.select('body').append('svg')
    .attr('width', width)
    .attr('height', height)
  .append('g')
    .attr('transform', 'translate(40,0)');

console.log(root);


var nodes = cluster.nodes(root),
    links = cluster.links(nodes);

var link = svg.selectAll('.link')
    .data(links)
  .enter().append('path')
    .attr('class', 'link')
    .attr('d', diagonal);

var node = svg.selectAll('.node')
    .data(nodes)
  .enter().append('g')
    .attr('class', 'node')
    .attr('transform', function(d) { return 'translate(' + d.y + ',' + d.x + ')'; })

node.append('circle')
    .attr('r', 4.5);

node.append('text')
    .attr('dx', function(d) { return d.children ? -8 : 8; })
    .attr('dy', 3)
    .style('text-anchor', function(d) { return d.children ? 'end' : 'start'; })
    .text(function(d) { return d.name; });


d3.select(self.frameElement).style('height', height + 'px');

</script>
"""


In [ ]:
display(HTML(visualizacao % (valores)))

### Visualização com árvore

In [ ]:
# Exemplo baseado no livro Mineração de Dados da Web Social
visualizacao = """<!DOCTYPE html>
<meta charset="utf-8">
<style>

.node circle { fill: #fff; stroke: steelblue; stroke-width: 1.5px; }
.node { font: 10px sans-serif; }
.link { fill: none; stroke: #ccc; stroke-width: 1.5px; }

</style>
<body>
<script src="https://d3js.org/d3.v3.min.js"></script>
<script>

var diameter = 960;

var tree = d3.layout.tree()
    .size([360, diameter / 2 - 120])
    .separation(function(a, b) { return (a.parent == b.parent ? 1 : 2) / a.depth; });

var diagonal = d3.svg.diagonal.radial()
    .projection(function(d) { return [d.y, d.x / 180 * Math.PI]; });

var svg = d3.select("body").append("svg")
    .attr("width", diameter)
    .attr("height", diameter - 150)
  .append("g")
    .attr("transform", "translate(" + diameter / 2 + "," + diameter / 2 + ")");

root = %s;

var nodes = tree.nodes(root),
    links = tree.links(nodes);

var link = svg.selectAll(".link")
    .data(links)
  .enter().append("path")
    .attr("class", "link")
    .attr("d", diagonal);

var node = svg.selectAll(".node")
    .data(nodes)
  .enter().append("g")
    .attr("class", "node")
    .attr("transform", function(d) { return "rotate(" + (d.x - 90) + ")translate(" + d.y + ")"; })

node.append("circle")
    .attr("r", 4.5);

node.append("text")
    .attr("dy", ".31em")
    .attr("text-anchor", function(d) { return d.x < 180 ? "start" : "end"; })
    .attr("transform", function(d) { return d.x < 180 ? "translate(8)" : "rotate(180)translate(-8)"; })
    .text(function(d) { return d.name; });


d3.select(self.frameElement).style("height", diameter - 150 + "px");

</script>
"""

In [ ]:
display(HTML(visualizacao % (valores)))

### Localização dos contatos

#### Teste com a API do Google Maps

In [ ]:
# https://console.developers.google.com
# https://developers.google.com/maps/documentation/geocoding/usage-and-billing

In [ ]:
!pip install geopy

In [ ]:
from geopy import geocoders

In [ ]:
g = geocoders.GoogleV3('')

In [ ]:
import pandas as pd
conexoes = pd.read_csv('conexoes.csv')
conexoes.drop(labels = ['Unnamed: 0'], axis = 1, inplace=True)
conexoes.head()

In [ ]:
localizacao = g.geocode('Google Canada')
localizacao

In [ ]:
localizacao.latitude, localizacao.longitude

#### Latitude e longitude dos contatos

In [ ]:
conexoes['Latitude'] = None
conexoes['Longitude'] = None
conexoes.head()

In [ ]:
for i in range(0, len(conexoes)):
  #print(conexoes['Company'][i])
  try:
    localizacao = g.geocode(conexoes['Company'][i])
  except:
    print(conexoes['Company'][i])

  if localizacao != None:
    conexoes['Latitude'][i] = localizacao.latitude
    conexoes['Longitude'][i] = localizacao.longitude

In [ ]:
conexoes

In [ ]:
conexoes.to_csv('conexoes_localizacao.csv')

In [ ]:
import pandas as pd
conexoes = pd.read_csv('conexoes_localizacao.csv')
conexoes.drop(labels = ['Unnamed: 0'], axis = 1, inplace=True)
conexoes.head()

#### Mapa de localizações com BaseMap

In [ ]:
!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [ ]:
conexoes['Latitude'].describe()

In [ ]:
conexoes['Longitude'].describe()

In [ ]:
lat1, lat2 = conexoes['Latitude'].min(), conexoes['Latitude'].max()
lon1, lon2 = conexoes['Longitude'].min(), conexoes['Longitude'].max()

In [ ]:
plt.figure(figsize=(10,10))
m = Basemap(projection='cyl', resolution='h',
            llcrnrlat = lat1, urcrnrlat = lat2,
            llcrnrlon = lon1, urcrnrlon = lon2)
m.drawcoastlines()
m.fillcontinents(color = 'palegoldenrod', lake_color = 'lightskyblue')
m.drawmapboundary(fill_color='lightskyblue')
m.scatter(conexoes['Longitude'], conexoes['Latitude'], s = 30, c = 'red', zorder = 2);

#### Obtenção dos países e cidades

In [ ]:
# https://github.com/thampiman/reverse-geocoder
!pip install reverse_geocoder

In [ ]:
import reverse_geocoder as rg

In [ ]:
localizacao = rg.search((conexoes['Latitude'][4], conexoes['Longitude'][4]))
localizacao

In [ ]:
localizacao[0]['cc'], localizacao[0]['name']

In [ ]:
conexoes['Pais'] = None
conexoes['Cidade'] = None
conexoes.head()

In [ ]:
for i in range(0, len(conexoes)):
  try:
    localizacao = rg.search((conexoes['Latitude'][i], conexoes['Longitude'][i]))
    conexoes['Pais'][i] = localizacao[0]['cc']
    conexoes['Cidade'][i] = localizacao[0]['name']
  except:
    print(conexoes['Company'][i])

In [ ]:
conexoes

In [ ]:
conexoes.to_csv('conexoes_localizacao_completo.csv')

In [ ]:
import pandas as pd
conexoes = pd.read_csv('conexoes_localizacao_completo.csv')
conexoes.drop(labels = ['Unnamed: 0'], axis = 1, inplace=True)
conexoes.head()

#### Gráfico por países e cidades

- Sigla dos países: http://manualdemarcas.inpi.gov.br/projects/manual-de-marcas-2-edicao-1-revisao/wiki/Siglas_de_pa%C3%ADses_e_organizações

In [ ]:
!pip install --upgrade plotly

In [ ]:
import plotly.express as px

In [ ]:
grafico = px.histogram(x = conexoes['Pais'], title = 'Contatos por país')
grafico.show()

In [ ]:
grafico = px.treemap(conexoes[conexoes['Pais'].notnull()], path = ['Pais', 'Cidade', 'Position', 'Full Name'])
grafico.show()

In [ ]:
grafico = px.histogram(x = conexoes['Cidade'], title = 'Contatos por cidade')
grafico.show()

In [ ]:
grafico = px.treemap(conexoes[conexoes['Cidade'].notnull()], path = ['Cidade', 'Position', 'Full Name'])
grafico.show()

In [ ]:
conexoes_us = conexoes[conexoes['Pais'] == 'US']
conexoes_us.shape

In [ ]:
conexoes.shape

In [ ]:
conexoes_us

In [ ]:
grafico = px.treemap(conexoes_us, path = ['Cidade', 'Position', 'Full Name'])
grafico.show()

#### Agrupamento de usuários por localização com k-means

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
conexoes.head(1)

In [ ]:
X = conexoes.iloc[:, 4:6].dropna().values
X

In [ ]:
kmeans = KMeans(n_clusters=15)
kmeans.fit(X)

In [ ]:
kmeans.labels_

In [ ]:
kmeans.cluster_centers_

#### Geração do arquivo para visualização no Google Earth

In [ ]:
!pip install simplekml

In [ ]:
import simplekml

In [ ]:
kml_contatos = simplekml.Kml()
for i in range(0, len(conexoes)):
  kml_contatos.newpoint(name = conexoes['Full Name'][i],
                        coords = [(conexoes['Longitude'][i], conexoes['Latitude'][i])])
kml_contatos.save('conexoes.kml')

In [ ]:
kml_grupos = simplekml.Kml()
for i in range(len(kmeans.cluster_centers_)):
  kml_grupos.newpoint(name = 'Grupo {}'.format(i),
                      coords = [(kmeans.cluster_centers_[i][1], kmeans.cluster_centers_[i][0])])
kml_grupos.save('grupos.kml')

## Convites

In [ ]:
import pandas as pd
convites = pd.read_csv('Invitations.csv')
convites.drop(labels = ['Unnamed: 0'], axis = 1, inplace=True)

In [ ]:
convites.head()

In [ ]:
convites['Latitude'] = None
convites['Longitude'] = None
convites['Pais'] = None
convites['Cidade'] = None
convites.head()

In [ ]:
from geopy import geocoders
g = geocoders.GoogleV3('')
for i in range(0, len(convites)):
  try:
    localizacao = g.geocode(convites['Company'][i])
  except:
    print(convites['Company'][i])

  if localizacao != None:
    convites['Latitude'][i] = localizacao.latitude
    convites['Longitude'][i] = localizacao.longitude

In [ ]:
convites.head()

In [ ]:
!pip install reverse_geocoder
import reverse_geocoder as rg

In [ ]:
for i in range(0, len(convites)):
  try:
    localizacao = rg.search((convites['Latitude'][i], convites['Longitude'][i]))
    convites['Pais'][i] = localizacao[0]['cc']
    convites['Cidade'][i] = localizacao[0]['name']
  except:
    print(convites['Company'][i])

In [ ]:
convites.head()

In [ ]:
convites.to_csv('convites_localizacao.csv')

In [ ]:
import pandas as pd
import plotly.express as px
convites = pd.read_csv('convites_localizacao.csv')
convites.drop(labels = ['Unnamed: 0'], axis = 1, inplace=True)
convites.head()

In [ ]:
!pip install --upgrade plotly

In [ ]:
grafico = px.treemap(convites[convites['Company'].notnull()], path = ['Company'])
grafico.show()

In [ ]:
convites2 = convites.loc[(convites['Pais'].notnull()) & (convites['Company'].notnull())]

In [ ]:
convites.shape, convites2.shape

In [ ]:
grafico = px.treemap(convites2, path=['Pais', 'Company', 'Cidade'])
grafico.show()

## Mensagens

### Carregamento da base de dados

In [ ]:
!pip install --upgrade plotly

In [ ]:
import pandas as pd
import plotly.express as px
mensagens = pd.read_csv('mensagens.csv')
mensagens.drop(labels = ['Unnamed: 0'], axis = 1, inplace=True)
mensagens

In [ ]:
mensagens.isnull().sum()

In [ ]:
mensagens['CONTENT'][446]

In [ ]:
mensagens['CONTENT'][4]

In [ ]:
mensagens.describe()

### Pré-processamento dos textos

In [ ]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
from bs4 import BeautifulSoup

In [ ]:
import re

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
nltk.download('stopwords')

In [ ]:
print(nltk.corpus.stopwords.words('portuguese'))

In [ ]:
len(nltk.corpus.stopwords.words('portuguese'))

In [ ]:
import string
string.punctuation

In [ ]:
def pre_processamento(texto):
  texto = texto.lower()
  texto = BeautifulSoup(texto, 'lxml').text
  texto = re.sub(r'https?://[A-Za-z0-9./]+', ' ', texto)
  tokens = []
  for token in nltk.word_tokenize(texto):
    tokens.append(token)
  #print(tokens)

  tokens = [palavra for palavra in tokens if palavra not in nltk.corpus.stopwords.words('portuguese') and palavra not in string.punctuation]
  #print(tokens)
  texto_formatado = ' '.join([str(elemento) for elemento in tokens])

  return texto_formatado

In [ ]:
pre_processamento(mensagens['CONTENT'][446] + 'https://www.iaexpert.academy')

### Pré-processamento da base de dados

In [ ]:
mensagens

In [ ]:
mensagens['CONTENT'] = mensagens['CONTENT'].apply(pre_processamento)

In [ ]:
mensagens

In [ ]:
mensagens['tamanho'] = mensagens['CONTENT'].apply(len)
mensagens

In [ ]:
mensagens['tamanho'].describe()

In [ ]:
len('olá jones robotic process automation rpa'.split())

In [ ]:
def conta_palavras(texto):
  return len(texto.split())

In [ ]:
mensagens['palavras'] = mensagens['CONTENT'].apply(conta_palavras)
mensagens

In [ ]:
mensagens['palavras'].describe()

In [ ]:
mensagens.shape

In [ ]:
mensagens = mensagens[mensagens['palavras'] > 5]
mensagens.shape

### Detecção do idioma

In [ ]:
import spacy

In [ ]:
!python3 -m spacy download pt

In [ ]:
!pip install spacy-langdetect

In [ ]:
from spacy_langdetect import LanguageDetector
pln = spacy.load('pt')
pln

In [ ]:
pln.add_pipe(LanguageDetector(), name='language_detector', last=True)
texto = 'Este é um texto em português'
doc = pln(texto)
print(doc._.language)

In [ ]:
texto = 'This is an english text'
doc = pln(texto)
print(doc._.language)

In [ ]:
doc._.language['language']

In [ ]:
mensagens = mensagens.reset_index(drop = True)

In [ ]:
mensagens['Idioma'] = None
for i in range(0, len(mensagens)):
  doc = pln(mensagens['CONTENT'][i])
  mensagens['Idioma'][i] = doc._.language['language']

In [ ]:
mensagens

In [ ]:
# Siglas: https://support.mozilla.org/pt-BR/kb/abreviacao-de-localizacao
grafico = px.histogram(x = mensagens['Idioma'])
grafico.show()

In [ ]:
mensagens_pt = mensagens[mensagens['Idioma'] == 'pt']
mensagens_pt.shape

In [ ]:
mensagens_en = mensagens[mensagens['Idioma'] == 'en']
mensagens_en.shape

### Nuvem de palavras

In [ ]:
textos = mensagens_pt['CONTENT'].tolist()
len(textos)

In [ ]:
print(textos[0:10])

In [ ]:
textos_string = ' '.join(textos)
textos_string

In [ ]:
len(textos_string)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(textos_string));

### Extração de entidades nomeadas

- Siglas: https://spacy.io/api/annotation#named-entities

In [ ]:
documento = pln(textos_string)

In [ ]:
from spacy import displacy
displacy.render(documento, style = 'ent', jupyter = True)

In [ ]:
for entidade in documento.ents:
  if entidade.label_ == 'LOC':
    print(entidade.text, entidade.label_)

### Classificação de sentimentos em inglês

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
classificador = SentimentIntensityAnalyzer()
classificador.polarity_scores('I love this food')

In [ ]:
classificador.polarity_scores('I hate this food')

In [ ]:
resultado = classificador.polarity_scores('I have this food')

In [ ]:
resultado

In [ ]:
resultado['neu']

In [ ]:
mensagens_en = mensagens_en.reset_index(drop=True)

In [ ]:
mensagens_en['Sentimento'] = None
for i in range(0, len(mensagens_en)):
  sentimento = classificador.polarity_scores(mensagens_en['CONTENT'][i])
  if sentimento['pos'] > sentimento['neg'] and sentimento['pos'] > sentimento['neu']:
    mensagens_en['Sentimento'][i] = 'Positivo'
  elif sentimento['neg'] > sentimento['pos'] and sentimento['neg'] > sentimento['neu']:
    mensagens_en['Sentimento'][i] = 'Negativo'
  elif sentimento['neu'] > sentimento['pos'] and sentimento['neu'] > sentimento['neg']:
    mensagens_en['Sentimento'][i] = 'Neutro'
  else:
    mensagens_en['Sentimento'][i] = 'Sem classificação'

In [ ]:
mensagens_en

### Classificação de sentimentos em português

#### Lematização

In [ ]:
def pre_processamento2(texto):
  documento = pln(texto)

  lista = []
  for token in documento:
    lista.append(token.lemma_)
  lista = ' '.join([str(elemento) for elemento in lista])
  return lista

In [ ]:
mensagens_pt = mensagens_pt.reset_index(drop=True)

In [ ]:
mensagens_pt['CONTENT'][4]

In [ ]:
pre_processamento2(mensagens_pt['CONTENT'][4])

In [ ]:
mensagens_pt.head(3)

In [ ]:
mensagens_pt['CONTENT'] = mensagens_pt['CONTENT'].apply(pre_processamento2)

In [ ]:
mensagens_pt.head(3)

#### Carregamento do classificador

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
classificador = spacy.load('/content/drive/MyDrive/Cursos - recursos/Mineração e Análise de Dados do LinkedIn/modelo')
classificador

#### Classificação

In [ ]:
mensagens_pt['CONTENT'][4]

In [ ]:
previsao = classificador(mensagens_pt['CONTENT'][4])
previsao.cats

In [ ]:
previsao.cats['POSITIVO']

In [ ]:
mensagens_pt['Sentimento'] = None
for i in range(0, len(mensagens_pt)):
  sentimento = classificador(mensagens_pt['CONTENT'][i])
  if sentimento.cats['POSITIVO'] > sentimento.cats['NEGATIVO']:
    mensagens_pt['Sentimento'][i] = 'Positivo'
  else:
    mensagens_pt['Sentimento'][i] = 'Negativo'

In [ ]:
mensagens_pt

In [ ]:
grafico = px.histogram(x = mensagens_pt['Sentimento'])
grafico.show()

In [ ]:
mensagens_pt[mensagens_pt['Sentimento'] == 'Negativo']